### 第(1)问代码

In [50]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

# 定义函数将经纬度转换为长度
def lat_lon_to_length(x, y, x_ref, y_ref):
    """
    将经纬度转换为长度。
    
    参数:
    x (float): 当前点的经度
    y (float): 当前点的纬度
    x_ref (float): 参考点的经度
    y_ref (float): 参考点的纬度
    
    返回:
    dx (float): 经度差转换为长度（米）
    dy (float): 纬度差转换为长度（米）
    """
    x1 = abs(x - x_ref)
    x2 = 360 - x1
    x3 = min(x1, x2)
    dx = 97.304 * x3 * 1000  # 转换为米
    dy = 111.263 * (y - y_ref) * 1000  # 转换为米
    return dx, dy

# 定义函数用于计算误差
def residuals(params, v, t_i, coords):
    """
    计算预测音爆抵达时间与实际记录时间之间的差值。
    
    参数:
    params (array): 包含 x, y, z, t0 的数组
    v (float): 音速（米/秒）
    t_i (array): 各监测设备记录的音爆抵达时间（秒）
    coords (array): 各监测设备的经纬度和高度
    
    返回:
    residuals (array): 各监测设备的误差数组
    """
    x, y, z, t0 = params
    residuals = []
    for i in range(len(t_i)):
        dx, dy = lat_lon_to_length(x, y, coords[i][0], coords[i][1])
        d_i = np.sqrt(dx**2 + dy**2 + (z - coords[i][2])**2)
        t_calc = t0 + d_i / v
        residuals.append(t_i[i] - t_calc)
    return residuals

# 音速
v = 340.0  # m/s

# 监测设备的坐标（经度，纬度，高程）
coords = np.array([
    [110.241, 27.204, 824],
    [110.780, 27.456, 727],
    [110.712, 27.785, 742],
    [110.251, 27.825, 850],
    [110.524, 27.617, 786],
    [110.467, 27.921, 678],
    [110.047, 27.121, 575]
])

# 检查经纬度范围
assert np.all((-180 <= coords[:, 0]) & (coords[:, 0] <= 180)), "经度范围应在-180°到180°之间"
assert np.all((-90 <= coords[:, 1]) & (coords[:, 1] <= 90)), "纬度范围应在-90°到90°之间"

# 音爆抵达时间
t_i = np.array([
    100.767,
    112.220,
    188.020,
    258.985,
    118.443,
    266.871,
    163.024
])

# 初始猜测值 (x, y, z, t0)
# 初始猜测值设为设备坐标的平均值和音爆时间的最小值
initial_guess = np.array([np.mean(coords[:, 0]), np.mean(coords[:, 1]), np.mean(coords[:, 2]), np.min(t_i)])

# 使用最小二乘法进行数值优化求解，并设置音爆时间的非负约束
bounds = ([-180, -90, -np.inf, -np.inf], [180, 90, np.inf, np.inf])  # 设置经度、纬度、高度和时间的范围
result = least_squares(residuals, initial_guess, bounds=bounds, args=(v, t_i, coords))

# 输出结果
x_sol, y_sol, z_sol, t0_sol = result.x
print(f"音爆发生的位置: 经度 = {x_sol:.6f}, 纬度 = {y_sol:.6f}, 高程 = {z_sol:.3f} 米")
print(f"音爆发生的时间: t0 = {t0_sol:.3f} 秒")

# 计算预测的音爆抵达时间
predicted_times = []
for i in range(len(t_i)):
    dx, dy = lat_lon_to_length(x_sol, y_sol, coords[i][0], coords[i][1])
    d_i = np.sqrt(dx**2 + dy**2 + (z_sol - coords[i][2])**2)
    t_calc = t0_sol + d_i / v
    predicted_times.append(t_calc)
    print(f"设备 {i+1} 的预测音爆抵达时间: {t_calc:.3f} 秒, 实际音爆抵达时间: {t_i[i]} 秒")

# 计算误差
errors = t_i - np.array(predicted_times)
print("误差:", errors)

音爆发生的位置: 经度 = 110.630992, 纬度 = 27.151860, 高程 = 963.253 米
音爆发生的时间: t0 = -6.536 秒
设备 1 的预测音爆抵达时间: 106.373 秒, 实际音爆抵达时间: 100.767 秒
设备 2 的预测音爆抵达时间: 101.746 秒, 实际音爆抵达时间: 112.22 秒
设备 3 的预测音爆抵达时间: 201.950 秒, 实际音爆抵达时间: 188.02 秒
设备 4 的预测音爆抵达时间: 239.127 秒, 实际音爆抵达时间: 258.985 秒
设备 5 的预测音爆抵达时间: 148.729 秒, 实际音爆抵达时间: 118.443 秒
设备 6 的预测音爆抵达时间: 249.501 秒, 实际音爆抵达时间: 266.871 秒
设备 7 的预测音爆抵达时间: 160.905 秒, 实际音爆抵达时间: 163.024 秒
误差: [ -5.60579127  10.47412431 -13.92987601  19.8576571  -30.28590803
  17.37036988   2.11942401]


### 算出7个设备为一组，每组算出全部时间组合。最后算出每个设备的误差累计值

In [51]:
# 数据重新识别
data = {
    "A": {"coords": (110.241, 27.204, 824), "times": [100.767, 164.229, 214.850, 270.065]},
    "B": {"coords": (110.783, 27.456, 727), "times": [92.453, 112.220, 169.362, 196.583]},
    "C": {"coords": (110.762, 27.785, 742), "times": [75.560, 110.696, 156.936, 188.020]},
    "D": {"coords": (110.251, 28.025, 850), "times": [94.653, 141.409, 196.517, 258.985]},
    "E": {"coords": (110.524, 27.617, 786), "times": [78.600, 86.216, 118.443, 126.669]},
    "F": {"coords": (110.467, 28.081, 678), "times": [67.274, 166.270, 175.482, 266.871]},
    "G": {"coords": (110.047, 27.521, 575), "times": [103.738, 163.024, 206.789, 210.306]}
}

# 提取所有设备的ID
device_ids = list(data.keys())

def dfs(path_times, path_indices, index, all_times_combinations, all_indices_combinations):
    # 如果路径长度等于设备数量，保存当前路径
    if len(path_times) == len(device_ids):
        all_times_combinations.append(list(path_times))
        all_indices_combinations.append(list(path_indices))
        return
    
    # 获取当前设备ID
    device_id = device_ids[index]
    
    # 对于当前设备的每个时间，进行深度优先搜索
    for i, time in enumerate(data[device_id]["times"]):
        path_times.append(time)
        path_indices.append(i + 1)  # 下标从1开始
        dfs(path_times, path_indices, index + 1, all_times_combinations, all_indices_combinations)
        path_times.pop()  # 回溯
        path_indices.pop()  # 回溯

# 初始化变量
all_times_combinations = []
all_indices_combinations = []

# 调用DFS
dfs([], [], 0, all_times_combinations, all_indices_combinations)

# 输出结果
print(f"总共有 {len(all_times_combinations)} 种方案。")

# 示例输出前5种方案及其对应的下标
# for i in range(5):
#     print(f"方案 {i + 1}: 时间 = {all_times_combinations[i]}, 下标 = {all_indices_combinations[i]}")
# 全部方案和下标已经存储在 all_times_combinations 和 all_indices_combinations 中

# 定义函数将经纬度转换为长度
def lat_lon_to_length(x, y, x_ref, y_ref):
    x1 = abs(x - x_ref)
    x2 = 360 - x1
    x3 = min(x1, x2)
    dx = 97.304 * x3 * 1000  # 转换为米
    dy = 111.263 * (y - y_ref) * 1000  # 转换为米
    return dx, dy

# 定义函数用于计算误差
def residuals(params, v, t_i, coords):
    x, y, z, t0 = params
    residuals = []
    for i in range(len(t_i)):
        dx, dy = lat_lon_to_length(x, y, coords[i][0], coords[i][1])
        d_i = np.sqrt(dx**2 + dy**2 + (z - coords[i][2])**2)
        t_calc = t0 + d_i / v
        residuals.append(t_i[i] - t_calc)
    return residuals

# 音速
v = 340.0  # m/s

# 监测设备的坐标（经度，纬度，高程）
coords = np.array([
    [110.241, 27.204, 824],
    [110.783, 27.456, 727],
    [110.762, 27.785, 742],
    [110.251, 28.025, 850],
    [110.524, 27.617, 786],
    [110.467, 28.081, 678],
    [110.047, 27.521, 575]
])

# 检查经纬度范围
assert np.all((-180 <= coords[:, 0]) & (coords[:, 0] <= 180)), "经度范围应在-180°到180°之间"
assert np.all((-90 <= coords[:, 1]) & (coords[:, 1] <= 90)), "纬度范围应在-90°到90°之间"

# 初始猜测值 (x, y, z, t0)
initial_guess = np.array([np.mean(coords[:, 0]), np.mean(coords[:, 1]), np.mean(coords[:, 2]), np.min(t_i)])

# 结果存储
results = []

# 初始化每个设备的累计误差值
S = np.zeros(len(coords))

# 对每种方案进行计算
for times in all_times_combinations:
    t_i = np.array(times)
    result = least_squares(residuals, initial_guess, bounds=([-180, -90, -np.inf, -np.inf], [180, 90, np.inf, np.inf]), args=(v, t_i, coords))
    x_sol, y_sol, z_sol, t0_sol = result.x
    predicted_times = []
    for i in range(len(t_i)):
        dx, dy = lat_lon_to_length(x_sol, y_sol, coords[i][0], coords[i][1])
        d_i = np.sqrt(dx**2 + dy**2 + (z_sol - coords[i][2])**2)
        t_calc = t0_sol + d_i / v
        predicted_times.append(t_calc)
    errors = t_i - np.array(predicted_times)
    results.append({
        "solution": (x_sol, y_sol, z_sol, t0_sol),
        "predicted_times": predicted_times,
        "errors": errors
    })
    # 累加每个设备的误差
    for i in range(len(errors)):
        S[i] += abs(errors[i])

# 示例输出前3种方案的结果
# for i in range(3):
#     res = results[i]
#     print(f"方案 {i + 1}:")
#     print(f"  音爆发生的位置: 经度 = {res['solution'][0]:.6f}, 纬度 = {res['solution'][1]:.6f}, 高程 = {res['solution'][2]:.3f} 米, 时间 = {res['solution'][3]:.3f} 秒")
#     for j in range(len(res['predicted_times'])):
#         print(f"  设备 {j + 1} 的预测音爆抵达时间: {res['predicted_times'][j]:.3f} 秒, 实际音爆抵达时间: {t_i[j]} 秒")
#     print(f"  误差: {res['errors']}")

# 输出每个设备的累计误差值
for i, device_id in enumerate(device_ids):
    print(f"设备 {device_id} 的累计误差值: {S[i]}")

print(f"总共有 {len(all_times_combinations)} 种方案。")

总共有 16384 种方案。
设备 A 的累计误差值: 411988.5392786633
设备 B 的累计误差值: 401951.07272401825
设备 C 的累计误差值: 459180.8282847572
设备 D 的累计误差值: 615772.1877287896
设备 E 的累计误差值: 225189.7247659522
设备 F 的累计误差值: 616722.714214588
设备 G 的累计误差值: 448227.07446855237
总共有 16384 种方案。


### 算出所有7个设备为一组的误差

In [52]:
import numpy as np
from scipy.optimize import least_squares
from itertools import combinations

# 数据重新识别
data = {
    "A": {"coords": (110.241, 27.204, 824), "times": [100.767, 164.229, 214.850, 270.065]},
    "B": {"coords": (110.783, 27.456, 727), "times": [92.453, 112.220, 169.362, 196.583]},
    "C": {"coords": (110.762, 27.785, 742), "times": [75.560, 110.696, 156.936, 188.020]},
    "D": {"coords": (110.251, 28.025, 850), "times": [94.653, 141.409, 196.517, 258.985]},
    "E": {"coords": (110.524, 27.617, 786), "times": [78.600, 86.216, 118.443, 126.669]},
    "F": {"coords": (110.467, 28.081, 678), "times": [67.274, 166.270, 175.482, 266.871]},
    "G": {"coords": (110.047, 27.521, 575), "times": [103.738, 163.024, 206.789, 210.306]}
}

# 提取所有设备的ID
device_ids = list(data.keys())

# 定义DFS函数生成所有可能的组合方案
def dfs(path_times, path_indices, index, all_times_combinations, all_indices_combinations, selected_devices):
    if len(path_times) == len(selected_devices):
        all_times_combinations.append(list(path_times))
        all_indices_combinations.append(list(path_indices))
        return
    
    device_id = selected_devices[index]
    
    for i, time in enumerate(data[device_id]["times"]):
        path_times.append(time)
        path_indices.append(i + 1)  # 下标从1开始
        dfs(path_times, path_indices, index + 1, all_times_combinations, all_indices_combinations, selected_devices)
        path_times.pop()
        path_indices.pop()

# 定义函数将经纬度转换为长度
def lat_lon_to_length(x, y, x_ref, y_ref):
    x1 = abs(x - x_ref)
    x2 = 360 - x1
    x3 = min(x1, x2)
    dx = 97.304 * x3 * 1000  # 转换为米
    dy = 111.263 * (y - y_ref) * 1000  # 转换为米
    return dx, dy

# 定义函数用于计算误差
def residuals(params, v, t_i, coords):
    x, y, z, t0 = params
    residuals = []
    for i in range(len(t_i)):
        dx, dy = lat_lon_to_length(x, y, coords[i][0], coords[i][1])
        d_i = np.sqrt(dx**2 + dy**2 + (z - coords[i][2])**2)
        t_calc = t0 + d_i / v
        residuals.append(t_i[i] - t_calc)
    return residuals

# 音速
v = 340.0  # m/s

# 初始化每个设备组合的累计误差值
combinations_errors = []

# 对所有7个设备的组合进行计算
for selected_devices in combinations(device_ids, 7):                                # 修改组数要改这里！！！
    # 生成这些设备的组合方案
    all_times_combinations = []
    all_indices_combinations = []
    dfs([], [], 0, all_times_combinations, all_indices_combinations, selected_devices)
    
    # 提取这些设备的坐标和时间
    coords = np.array([data[device]["coords"] for device in selected_devices])
    
    # 初始化当前组合的误差值
    now = 0
    
    # 对每种方案进行计算
    for times in all_times_combinations:
        t_i = np.array(times)
        initial_guess = np.array([np.mean(coords[:, 0]), np.mean(coords[:, 1]), np.mean(coords[:, 2]), np.min(t_i)])
        result = least_squares(residuals, initial_guess, bounds=([-180, -90, -np.inf, -np.inf], [180, 90, np.inf, np.inf]), args=(v, t_i, coords))
        x_sol, y_sol, z_sol, t0_sol = result.x
        predicted_times = []
        for i in range(len(t_i)):
            dx, dy = lat_lon_to_length(x_sol, y_sol, coords[i][0], coords[i][1])
            d_i = np.sqrt(dx**2 + dy**2 + (z_sol - coords[i][2])**2)
            t_calc = t0_sol + d_i / v
            predicted_times.append(t_calc)
        errors = t_i - np.array(predicted_times)
        
        # 累加当前组合的误差
        now += np.sum(np.abs(errors))
    
    # 记录当前组合的累计误差值
    combinations_errors.append((selected_devices, now))

# 排序组合误差值，从小到大
combinations_errors.sort(key=lambda x: x[1])

# 输出每个设备组合的累计误差值
for devices, error in combinations_errors:
    devices_str = ", ".join(devices)
    print(f"设备组合 ({devices_str}) 的累计误差值: {error}")

设备组合 (A, B, C, D, E, F, G) 的累计误差值: 3179037.657807793


### 算出所有7个设备为一组的最优方案

In [1]:
import numpy as np
from scipy.optimize import least_squares
from itertools import combinations, product
import matplotlib.pyplot as plt

# 数据重新识别
data = {
    "A": {"coords": (110.241, 27.204, 824), "times": [100.767, 164.229, 214.850, 270.065]},
    "B": {"coords": (110.783, 27.456, 727), "times": [92.453, 112.220, 169.362, 196.583]},
    "C": {"coords": (110.762, 27.785, 742), "times": [75.560, 110.696, 156.936, 188.020]},
    "D": {"coords": (110.251, 28.025, 850), "times": [94.653, 141.409, 196.517, 258.985]},
    "E": {"coords": (110.524, 27.617, 786), "times": [78.600, 86.216, 118.443, 126.669]},
    "F": {"coords": (110.467, 28.081, 678), "times": [67.274, 166.270, 175.482, 266.871]},
    "G": {"coords": (110.047, 27.521, 575), "times": [103.738, 163.024, 206.789, 210.306]}
}

# 提取所有设备的ID
device_ids = list(data.keys())

# 定义函数将经纬度转换为长度
def lat_lon_to_length(x, y, x_ref, y_ref):
    x1 = abs(x - x_ref)
    x2 = 360 - x1
    x3 = min(x1, x2)
    dx = 97.304 * x3 * 1000  # 转换为米
    dy = 111.263 * (y - y_ref) * 1000  # 转换为米
    return dx, dy

# 定义函数用于计算误差
def residuals(params, v, t_i, coords):
    x, y, z, t0 = params
    residuals = []
    for i in range(len(t_i)):
        dx, dy = lat_lon_to_length(x, y, coords[i][0], coords[i][1])
        d_i = np.sqrt(dx**2 + dy**2 + (z - coords[i][2])**2)
        t_calc = t0 + d_i / v
        residuals.append(t_i[i] - t_calc)
    return residuals

# 音速
v = 340.0  # m/s

# 定义主函数，用于计算最佳组合
def find_best_combination():
    for selected_devices in combinations(device_ids, 7):                               # 修改组数要改这里！！！
        print(f"正在处理设备组合: {selected_devices}")  # 调试信息
        
        # 生成这些设备的时间组合
        time_combinations = list(product(*[data[device]["times"] for device in selected_devices]))
        
        # 调试信息，确保时间组合正确生成
        # print(f"生成的时间组合数量: {len(time_combinations)}")
        # for comb in time_combinations[:5]:  # 只打印前5个组合
        #     print(f"时间组合: {comb}")
        
        # 提取这些设备的坐标和时间
        coords = np.array([data[device]["coords"] for device in selected_devices])
        
        # 初始化误差列表
        results = []
        
        # 对每种方案进行计算
        for times in time_combinations:
            t_i = np.array(times)
            initial_guess = np.array([np.mean(coords[:, 0]), np.mean(coords[:, 1]), np.mean(coords[:, 2]), np.min(t_i)])
            result = least_squares(residuals, initial_guess, bounds=([-180, -90, 0, -np.inf], [180, 90, np.inf, np.inf]), args=(v, t_i, coords))
            x_sol, y_sol, z_sol, t0_sol = result.x
            predicted_times = []
            for i in range(len(t_i)):
                dx, dy = lat_lon_to_length(x_sol, y_sol, coords[i][0], coords[i][1])
                d_i = np.sqrt(dx**2 + dy**2 + (z_sol - coords[i][2])**2)
                t_calc = t0_sol + d_i / v
                predicted_times.append(t_calc)
            errors = t_i - np.array(predicted_times)
            # 调试信息
            # print(f"计算中间结果: 设备组合 = {selected_devices}, times = {times}, t0_sol = {t0_sol}, predicted_times = {predicted_times}, errors = {errors}")
            # 计算当前组合的误差之和
            total_error = np.sum(np.abs(errors))
            results.append((times, (x_sol, y_sol, z_sol), t0_sol, total_error))
        
        # 按误差排序
        results.sort(key=lambda x: x[3])
        
        # DFS + 剪枝寻找最佳组合
        def find_combination_with_dfs():
            best_combination = []
            best_error_sum = float('inf')
            
            def dfs(path, times_path, errors_path, used_times, idx):
                nonlocal best_combination, best_error_sum
                
                if len(path) == 4:
                    error_sum = sum(errors_path)
                    if error_sum < best_error_sum:
                        best_error_sum = error_sum
                        best_combination = list(path)
                    return
                
                if idx >= len(results):
                    return
                
                for i in range(idx, len(results)):
                    times, coords_sol, t, err = results[i]
                    # 检查当前组合的时间是否在已使用的时间列表中
                    if any(time in used_times for time in times):
                        continue
                    # 检查预测时间差是否大于5秒
                    if path and (max(times_path + [t]) - min(times_path + [t]) > 5):
                        continue
                    # 检查当前路径的误差和是否已经超过当前最佳误差和
                    if sum(errors_path) + err >= best_error_sum:
                        continue
                    dfs(path + [results[i]], times_path + [t], errors_path + [err], used_times + list(times), i + 1)
            
            dfs([], [], [], [], 0)
            
            return best_combination, best_error_sum
        
        best_combination, best_error_sum = find_combination_with_dfs()
        
        # 输出结果
        if best_combination:
            print(f"设备组合: {', '.join(selected_devices)}")
            for i, (times, coords_sol, t, err) in enumerate(best_combination):
                print(f"组合 {i + 1}: Times = {times}, Predicted coords = {coords_sol}, Predicted Time = {t:.3f}, Error Sum = {err:.3f}")
            print(f"最小误差之和: {best_error_sum:.3f}\n")
        else:
            print('Can not find solution!')

# 运行主函数
find_best_combination()

正在处理设备组合: ('A', 'B', 'C', 'D', 'E', 'F', 'G')
